In [1]:
!pip3 install l2rpn_baselines
import grid2op

     |████████████████████████████████| 174kB 28.8MB/s 
     |████████████████████████████████| 11.7MB 10.8MB/s 
     |████████████████████████████████| 9.5MB 29.2MB/s 
     |████████████████████████████████| 5.8MB 22.7MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 153kB 44.9MB/s 
     |████████████████████████████████| 3.2MB 21.2MB/s 
  Created wheel for l2rpn-baselines: filename=l2rpn_baselines-0.5.1-cp37-none-any.whl size=246282 sha256=65088dc848893d54b54c72ef17a815b2ee537008102b744ff83438c1ccf7c3a7
  Stored in directory: /root/.cache/pip/wheels/ba/c0/fc/2d79824b69e903348a57ddbbb841b4ec2d9c3ebe936d2177f7
  Created wheel for grid2op: filename=Grid2Op-1.5.2-cp37-none-any.whl size=11953700 sha256=49aff9090d96a7031369715c100ca4240d6b6e0e1913524626ace405cb10657b
  Stored in directory: /root/.cache/pip/wheels/c1/a2/11/ae011868969f9997cc4fd23997fa55671cd49297986c40e7f6
  Created wheel for pandapower: filename=pandapower-2.6.0-cp37-none-a

In [ ]:
env = grid2op.make()

/usr/local/lib/python3.7/dist-packages/grid2op/MakeEnv/Make.py:329: UserWarning: It is the first time you use the environment "rte_case14_realistic".
We will attempt to download this environment from remote
  warnings.warn(_MAKE_FIRST_TIME_WARN.format(dataset_name))
rte_case14_realistic.tar.bz2: 0.00B [00:00, ?B/s]

downloading the training data, this may take a while.


rte_case14_realistic.tar.bz2: 292MB [00:20, 14.2MB/s]                           


Extract the tar archive in "/root/data_grid2op"
You may now use the environment "rte_case14_realistic" with the available data by invoking:
	env = grid2op.make("rte_case14_realistic")


In [ ]:
from l2rpn_baselines.DuelQLeapNet.LeapNet_NNParam import LeapNet_NNParam

In [ ]:
sizes = [512, 512, 256, 256]

In [ ]:
li_attr_obs_X = ["day_of_week", "hour_of_day", "minute_of_hour", "prod_p", "prod_v", "load_p", "load_q",
                     "actual_dispatch", "target_dispatch"]

In [ ]:
x_dim = LeapNet_NNParam.get_obs_size(env, li_attr_obs_X)

In [ ]:
li_attr_obs_Tau = ["topo_vect", "time_before_cooldown_line", "time_before_cooldown_sub",
                       "timestep_overflow", "line_status", "rho"]

In [ ]:
tau_dims = [LeapNet_NNParam.get_obs_size(env, [el]) for el in li_attr_obs_Tau]

In [ ]:
 kwargs_archi = {'sizes': sizes,
                        'activs': ["relu" for _ in sizes],
                        'x_dim': x_dim,
                        'tau_dims': tau_dims,
                        'tau_adds': [0.0 for _ in range(len(tau_dims))],  # add some value to taus
                        'tau_mults': [1.0 for _ in range(len(tau_dims))],  # divide by some value for tau (after adding)
                        "list_attr_obs": li_attr_obs_X,
                        "list_attr_obs_tau": li_attr_obs_Tau
                        }

In [ ]:
from l2rpn_baselines.DuelQLeapNet import train
train(env, iterations=50, kwargs_archi=kwargs_archi, save_path='./leapnet')

100%|██████████| 50/50 [00:04<00:00, 10.06it/s]


In [ ]:
from l2rpn_baselines.DuelQLeapNet import evaluate
eval_tuple = evaluate(env, nb_episode=2, verbose=False, load_path='./leapnet', max_steps=5)

In [ ]:
eval_tuple

(<l2rpn_baselines.DuelQLeapNet.DuelQLeapNet.DuelQLeapNet at 0x7ffa988a24d0>,
 [('/root/data_grid2op/rte_case14_realistic/chronics/000',
   '000',
   2162.677490234375,
   3,
   5),
  ('/root/data_grid2op/rte_case14_realistic/chronics/001',
   '001',
   4364.4326171875,
   5,
   5)])

In [ ]:
eval_tuple[1][0][2] # This is the cumulative reward in that round
eval_tuple[1][1][2] # This is the cumulative reward in that round

4364.4326171875

In [ ]:
eval_tuple[0]